Menambahkan library yang diperlukan

In [1]:
import pandas as pd
import tensorflow as tf
import requests
import json
import base64

2024-11-06 22:18:24.663245: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 22:18:24.708752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Selanjutnya mendefiniskan function untuk melakukan konversi data ke dalam format `tf.train.Feature` dengan masing-masing tipe data yang dimiliki.

In [2]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

Function `prepare_json()` digunakan untuk melakukan konversi data ke dalam format yang sesuai dengan dimiliki oleh inputan model.

In [3]:
def prepare_json(inputs):
    feature_spec = dict()
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    result = [{
        "examples": {
            "b64": base64.b64encode(example).decode()
        }   
    }]
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

Selanjutnya function `predict()` digunakan untuk melakukan request prediksi ke dalam model yang telah di-deploy sebelumnya.

In [4]:
def predict(inputs):
    url = "http://localhost:8080/v1/models/customer-churn-model:predict"
    headers = {"content-type": "application/json"}
    data = []
    for i in range(len(inputs['Age'])):
        data.append(prepare_json({key: value[i] for key, value in inputs.items()}))
    
    responses = []
    for i in range(len(inputs['Age'])):
        response = requests.post(url, data=data[i], headers=headers)
        response = 'Churn' if response.json()['predictions'][0][0] > 0.5 else 'Not Churn'
        responses.append(response)
    return responses


Mengambil beberapa data secara acak dari dataset eval yang digunakan untuk melakukan prediksi.

In [6]:
df_data = pd.read_csv("data/customer_churn_dataset-testing-master.csv")
df_data.pop("Churn")
df_data = df_data.sample(5)
inputs = {key: value for key, value in df_data.to_dict(orient="list").items()}

prediction = predict(inputs)
print(prediction)

['Churn', 'Churn', 'Churn', 'Churn', 'Not Churn']
